In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Stock Price Prediction for RELIND stock
The first thing I did after I recieved my first salary? I invested a 10K directly to the RELIND stock and well...... I faced loss. 
So well, in this notebook, I would want to perhaps try and predict the future?

In [ ]:
import numpy as np
import pandas as pd

import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pandas_datareader as web
import datetime as dt
plt.style.use('ggplot') 
# Above is a special style template for matplotlib, highly useful for visualizing time series data
%matplotlib inline
from plotly import tools
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
df=pd.read_csv('../input/reliance-nse-stock-data/RILO - Copy.csv')
df.head()

## PART 1 - EDA

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
#Check for Null Values
df.isnull().sum().sum()

In [ ]:
# Plot the Closing Point Figure
fig = go.Figure()

fig = make_subplots(rows=2, cols=1,subplot_titles=("RELIND NSE Candlestick Chart","RELIND NSE Volume"))
fig.append_trace(go.Scatter(x=df['Date'], y=df['Close Price'],
                    mode='lines',
                    name='lines'),row=1,col=1)

fig.append_trace(go.Scatter(x=df['Date'], y=df['Total Traded Quantity'],
                    mode='lines',
                    name='lines'),row=2,col=1)

fig.update_layout(title="RELIND NSE Exploratory Data Analysis",
    xaxis_title="Price",
        yaxis_title="Time Period",height=800)


fig.show()

In [ ]:
# Plot the Candlestick Chart
fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open Price'],
                high=df['High Price'],
                low=df['Low Price'],
                close=df['Close Price'])])
fig.update_layout(title="RELIND NSE Candlestick Graph",
    xaxis_title="Price",
        yaxis_title="Time Period")

fig.show()

In [ ]:
#Create Rolling Means for The Stock Data
tenDay = df['Close Price'].rolling(10).mean()
twentyDay = df['Close Price'].rolling(20).mean()
fiftyDay = df['Close Price'].rolling(50).mean()
EMA20Day = df['Close Price'].ewm(span=20,adjust=False).mean()
EMA200Day = df['Close Price'].ewm(span=200,adjust=False).mean()

df['tenDay'] = tenDay
df['twentyDay'] = twentyDay
df['fiftyDay'] = fiftyDay
df['EMA20Day'] = EMA20Day
df['EMA200Day'] = EMA200Day


fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=df['Date'], y=df['Close Price'], name='Actual Close Price',
                         line=dict(color='#489ac7', width=1.5)))
fig.add_trace(go.Scatter(x=df['Date'], y=df['tenDay'], name='Ten Day Rolling Mean ',
                         line=dict(color='#f79763', width=1)))
fig.add_trace(go.Scatter(x=df['Date'], y=df['twentyDay'], name='Twenty Day Rolling Mean ',
                         line=dict(color='#b563f7', width=1)))
fig.add_trace(go.Scatter(x=df['Date'], y=df['fiftyDay'], name='Fifty Day Rolling Mean ',
                         line=dict(color='#f76386', width=1)))
fig.add_trace(go.Scatter(x=df['Date'], y=df['EMA20Day'], name='EMA Twenty Days ',
                         line=dict(color='#3bffa0', width=2)))
fig.add_trace(go.Scatter(x=df['Date'], y=df['EMA200Day'], name='EMA TwoHundred Days ',
                         line=dict(color='#36074a', width=1.5)))

fig.update_layout(title="RELIND NSE Moving Averages",
    xaxis_title="Price",
        yaxis_title="Time Period")

In [ ]:
#draw a quick correlation plot (drop time as a feature)
#also realise that it does not provide much information, now that we already know it is a time series
correlation_df = df.copy()
correlation_df = correlation_df.drop(columns =['Symbol','Series','Date'])
corr = correlation_df.corr()


fig = go.Figure(data=[go.Heatmap(z=corr.values,
                  x=corr.index.values,
                  y=corr.columns.values)])
fig.update_layout(title="Correlation Heatmap",height=600,width=800)
fig.show()

In [ ]:
# Partial Auto Correlation Plot
plot_pacf(df["Close Price"],lags=45,title="Partial AutoCorrelation Plot with RELIND NSE Close Price")
plt.figure()
plt.show()

## PART 2 - Time Series Analysis

In [ ]:
# Augmented Dickey Fuller Test, in order to check if the time series is stationary.
# Null Hypothesis means that the data is not stationary.
# Alternate Hypothesis means that the data is stationary 
def stationaryTest(dataframe):
    adft = adfuller(dataframe,autolag='AIC')
    labelsADF= ['Test Statistics','p-value','No. of lags used','Number of observations used']
    for values,labels in zip(adft,labelsADF):
        print (labels + " : " + str(values))
    if (adft[1]<= 0.05):
        print("Reject the Null Hypothesis. Data is Stationary")
    else:
        print("Accept the Null Hypothesis. The TS is not stationary")
        
stationaryTest(df['Close Price'])

In [ ]:
# Decompose Seasonality from the Datas
result = seasonal_decompose(df[['Close Price']], model='multiplicative', freq = 45)
fig = plt.figure()  
fig = result.plot()  
fig.set_size_inches(15, 15)

In [ ]:
# Check if the Residual Value is Stationary, if yes then the period
df['residualAfterDecomposition']= result.resid
stationaryTest(df['residualAfterDecomposition'].dropna())